# Import

In [ ]:
import pandas as pd
import json

from curation_tools.curation_tools import (
    CuratedDataset,
    ObsSchema,
    VarSchema,
    Experiment,
    download_file,
    upload_parquet_to_bq
)

import logging
logging.basicConfig(
    level=logging.DEBUG,
    format="%(asctime)s %(levelname)s %(name)s: %(message)s",
    handlers=[
        logging.FileHandler("curation.log"),
        logging.StreamHandler(),  # keep console output too
    ],
    force=True,
)

# Download data

In [ ]:
noncurated_path = "../non_curated/h5ad/replogle_2022_k562_gw_normalized.h5ad"
download_file(
    url="https://plus.figshare.com/ndownloader/files/35774440",
    dest_path=noncurated_path
)

# Initialise the dataset object

In [ ]:
cur_data = CuratedDataset(
    obs_schema=ObsSchema,
    var_schema=VarSchema,
    exp_metadata_schema=Experiment,
    noncurated_path=noncurated_path
)

cur_data.load_data()

In [ ]:
cur_data.adata.obs

# OBS slot curation

### Show unique perturbations

In [ ]:
cur_data.show_unique(slot = 'obs', column = 'sgID_AB')

### Rename `sgID_AB` to `perturbation_name`

In [ ]:
cur_data.rename_columns(slot = 'obs', name_dict = {'sgID_AB': 'perturbation_name'})

### Add guide RNA information

In [ ]:
# download the guide RNA spreadsheet
download_file(
    url="https://ars.els-cdn.com/content/image/1-s2.0-S0092867422005979-mmc1.xlsx",
    dest_path="../supplementary/replogle_2022_guide_info.xlsx"
)

# read in the guide RNA spreadsheet
# guides for the K562 essential day 6 library are in "TabB_K562_day6_library"
guide_info_df = pd.read_excel("../supplementary/replogle_2022_guide_info.xlsx", sheet_name="TabA_K562_day8_library")

# create perturbation_name column in guide_info_df
guide_info_df['perturbation_name'] = guide_info_df['sgID_A'] + '|' + guide_info_df['sgID_B']
# replace commas with hyphens in perturbation_name
guide_info_df['perturbation_name'] = guide_info_df['perturbation_name'].str.replace(',', '-')
# check that all perturbation names in cur_data are in guide_info_df
print(f"All perturbation names in cur_data are in guide_info_df: {cur_data.adata.obs['perturbation_name'].isin(guide_info_df['perturbation_name']).all()}")
# create guide_sequence column in guide_info_df
guide_info_df['guide_sequence'] = guide_info_df['targeting sequence A'] + '|' + guide_info_df['targeting sequence B']
# subset for necessary columns
guide_info_df = guide_info_df[['perturbation_name', 'guide_sequence']]
# merge cur_data.adata.obs with guide_info_df on perturbation_name
cur_data.adata.obs = cur_data.adata.obs.merge(guide_info_df, on='perturbation_name', how='left')
# check that there are no missing guide sequences
print(f"Number of missing guide sequences: {cur_data.adata.obs['guide_sequence'].isna().sum()}")


### Standardise perturbation targets

In [ ]:
cur_data.standardize_genes(
    slot='obs',
    input_column='gene',
    input_column_type='gene_symbol',
    multiple_entries=False
)

### Add `perturbed_target_number` column

In [ ]:
cur_data.count_entries(
    slot='obs',
    input_column='perturbed_target_symbol',
    count_column_name='perturbed_target_number',
    sep='|'
)

### Encode chromosomes as integers

In [ ]:
cur_data.chromosome_encoding()

In [ ]:
cur_data.show_obs(['perturbation_name', 'perturbed_target_chromosome_encoding'])

### Add metadata

In [ ]:
cur_data.create_columns(
    overwrite=True,
    slot="obs",
    col_dict={
        "dataset_id": cur_data.dataset_id,
        "sample_id": range(1, cur_data.adata.obs.shape[0] + 1),
        # perturbation type
        "perturbation_type_label": "CRISPRi",
        "perturbation_type_id": None,
        "data_modality": "CRISPR screen",
        "significant": None,
        "significance_criteria": None,
        "score_interpretation": None,

        # treatment
        "treatment_label": None,
        "treatment_id": None,
        # model system
        "model_system_label": "cell_line",
        "model_system_id": None,
        "tissue": "blood",
        "cell_line_label": "K 562 cell",
        "cell_type_label": "lymphoblast",
        "disease_label": "chronic myelogenous leukemia, BCR-ABL1 positive",

        "timepoint": "P8DT0H0M0S",
        "species": "Homo sapiens",
        "sex_label": "female",
        "sex_id": None,
        "developmental_stage_label": "adult",
        "developmental_stage_id": None,

        "study_title": "Mapping information-rich genotype-phenotype landscapes with genome-scale Perturb-seq",
        "study_uri": "https://doi.org/10.1016/j.cell.2022.05.013",
        "study_year": 2022,
        "first_author": "Joseph M Replogle",
        "last_author": "Jonathan S Weissman",

        "experiment_title": "K562 day 8 essential-scale Perturb-seq experiment",
        "experiment_summary": """
            K562 chronic myeloid leukemia cells were transduced with a sgRNA library targeting genes expressed in K562 cells, transcription factors as detailed in Lambert et al. (2018) and Cancer Dependency Map common essential genes.
            A total of 9,866 genes were targeted.
            Cells were sampled at day 8 after lentiviral transduction.
            Multiplexed CRISPRi library containing two distinct guides targeting the same gene were used.
            """,

        "number_of_perturbed_targets": len(set(cur_data.adata.obs['perturbed_target_coord'])),
        "number_of_perturbed_samples": cur_data.adata.obs.shape[0],

        "library_generation_type_id": "EFO:0022868",
        "library_generation_type_label": "endogenous",

        "library_generation_method_id": None,
        "library_generation_method_label": "dCas9-KRAB",

        "enzyme_delivery_method_id": None,
        "enzyme_delivery_method_label": "lentivirus transduction",

        "library_delivery_method_id": None,
        "library_delivery_method_label": "lentivirus transduction",

        "enzyme_integration_state_id": None,
        "enzyme_integration_state_label": "random locus integration",

        "library_integration_state_id": None,
        "library_integration_state_label": "random locus integration",

        "enzyme_expression_control_id": None,
        "enzyme_expression_control_label": "constitutive transgene expression",

        "library_expression_control_id": None,
        "library_expression_control_label": "constitutive transgene expression",

        "library_name": "custom",
        "library_uri": None,

        "library_format_id": None,
        "library_format_label": "pooled",

        "library_scope_id": None,
        "library_scope_label": "focused",

        "library_perturbation_type_id": None,
        "library_perturbation_type_label": "inhibition",

        "library_manufacturer": "Weissman Lab",
        "library_lentiviral_generation": "3",
        "library_grnas_per_target": "2",
        "library_total_grnas": str(cur_data.adata.obs['guide_sequence'].str.split('|').explode().nunique()),
        "library_total_variants": None,

        "readout_dimensionality_id": None,
        "readout_dimensionality_label": "high-dimensional assay",

        "readout_type_id": None,
        "readout_type_label": "transcriptomic",

        "readout_technology_id": None,
        "readout_technology_label": "single-cell rna-seq",

        "method_name_id": None,
        "method_name_label": "Perturb-seq",

        "method_uri": None,

        "sequencing_library_kit_id": None,
        "sequencing_library_kit_label": "10x Genomics Single Cell 3-prime v3",

        "sequencing_platform_id": None,
        "sequencing_platform_label": "Ultima Genomics UG100",

        "sequencing_strategy_id": None,
        "sequencing_strategy_label": "barcode sequencing",

        "software_counts_id": None,
        "software_counts_label": "CellRanger",

        "software_analysis_id": None,
        "software_analysis_label": "custom",

        "reference_genome_id": None,
        "reference_genome_label": "GRCh38",
        
        "license_label": "CC BY 4.0",
        "license_id": "SWO:1000065",

        "associated_datasets": json.dumps([
            {
                "dataset_accession": "K562_gwps_raw_bulk_01",
                "dataset_uri": "https://plus.figshare.com/ndownloader/files/35774443",
                "dataset_description": "Raw, pseudo-bulk expression data for genes expressed at >0.01 UMI per cell",
                "dataset_file_name": "K562_gwps_raw_bulk_01.h5ad",
            },
            {
                "dataset_accession": "K562_gwps_normalized_bulk_01",
                "dataset_uri": "https://plus.figshare.com/ndownloader/files/35773217",
                "dataset_description": "Gemgroup Z-normalized pseudo-bulk expression data for genes expressed at >0.01 UMI per cell",
                "dataset_file_name": "K562_gwps_normalized_bulk_01.h5ad",
            },
            {
                "dataset_accession": "K562_gwps_raw_singlecell_01",
                "dataset_uri": "https://plus.figshare.com/ndownloader/files/35775507",
                "dataset_description": "Raw, single-cell expression data for genes expressed at >0.01 UMI per cell",
                "dataset_file_name": "K562_gwps_raw_singlecell_01.h5ad",
            },
            {
                "dataset_accession": "K562_gwps_normalized_singlecell_01",
                "dataset_uri": "https://plus.figshare.com/ndownloader/files/35774440",
                "dataset_description": "Gemgroup Z-normalized single-cell expression data for genes expressed at >0.01 UMI per cell",
                "dataset_file_name": "K562_gwps_normalized_singlecell_01.h5ad",
            }
        ])
    }
)

In [ ]:
cur_data.adata.obs

### Curate tissue information


In [ ]:
cur_data.standardize_ontology(
    input_column='tissue',
    column_type='term_name',
    ontology_type='tissue',
    overwrite=True
)

### Curate cell type information

In [ ]:
cur_data.standardize_ontology(
    input_column='cell_type_label',
    column_type='term_name',
    ontology_type='cell_type',
    overwrite=True
)

### Curate cell line information

In [ ]:
cur_data.standardize_ontology(
    input_column='cell_line_label',
    column_type='term_name',
    ontology_type='cell_line',
    overwrite=True
)

### Curate disease information

In [ ]:
cur_data.standardize_ontology(
    input_column='disease_label',
    column_type='term_name',
    ontology_type='disease',
    overwrite=True
)

### Match schema column order

In [ ]:
cur_data.match_schema_columns(slot='obs')

### Validate obs metadata

In [ ]:
cur_data.validate_data(slot='obs', verbose=True)

In [ ]:
cur_data.adata.obs[['library_total_grnas', 'library_total_variants']]

In [ ]:
cur_data.show_obs(['perturbation_name', 'perturbed_target_symbol', 'perturbed_target_ensg', 'perturbed_target_coord'])

# VAR slot curation

### Standardise genes

In [ ]:
cur_data.show_var()

In [ ]:
cur_data.create_columns(
    slot = 'var',
    col_dict={'gene_ensembl_id': cur_data.adata.var.index},
    overwrite=True
)

In [ ]:
cur_data.standardize_genes(
    slot="var",
    input_column="gene_ensembl_id",
    input_column_type="ensembl_gene_id",
    remove_version=False,
    multiple_entries=False
)

### Validate var metadata

In [ ]:
cur_data.validate_data(slot='var')

# Save the dataset

In [ ]:
cur_data.save_curated_data_h5ad()

In [ ]:
cur_data.save_curated_data_parquet(split_metadata=True, save_metadata_only=True)

# Upload to BigQuery

In [ ]:
upload_parquet_to_bq(
    parquet_path='../curated/parquet/replogle_2022_k562_gw_normalized_curated_metadata.parquet',
    bq_dataset_id='prj-ext-dev-pertcat-437314.perturb_seq',
    bq_table_name='metadata',
    key_columns=['dataset_id', 'sample_id'],
    verbose=True
)

# Upload to GC Storage

In [ ]:
!gcloud storage cp /content/PerturbationCatalogue/curated/h5ad/replogle_2022_k562_gw_normalized_curated.h5ad gs://perturbation-catalogue-lake/perturbseq/curated/

In [ ]:
!gcloud storage cp /content/PerturbationCatalogue/curated/h5ad/replogle_2022_k562_essential_normalized_curated.h5ad gs://perturbation-catalogue-lake/perturbseq/curated/

In [ ]:
!gcloud storage cp /content/PerturbationCatalogue/curated/h5ad/replogle_2022_rpe1_essential_normalized_curated.h5ad gs://perturbation-catalogue-lake/perturbseq/curated/